In [ ]:
import math

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#  sample a dataset
x = np.random.uniform(-1, 1, size=1000)
x = np.expand_dims(x, 1)

#  change y_true to get the net to fit a different function
y_true = 10 * np.power(x, 3) + np.power(x, 2) + 2 * np.power(x, 2)
noise = np.random.normal(0, 1, size=x.shape[0]) * 0.5
noise = np.expand_dims(noise, 1)
y = y_true + noise

assert y_true[0] + noise[0] == y[0]
assert x.shape[0] == y.shape[0]

In [ ]:
#  network architecture

#  input
i_size = 1
#  hidden layer
h_size = 10
activation = 'sigmoid'  #  relu or sigmoid
#  output layer
o_size = 1

#  hyperparameters
lr = 0.0001
reg = 0.01

In [ ]:
#  sample weights from a normal distribution, biases at 0
w0 = np.random.randn(i_size, h_size)
b0 = np.zeros((1, h_size))

w1 = np.random.randn(h_size, o_size)
b1 = np.zeros((1, o_size))

w = [w0, w1]
b = [b0, b1]

In [ ]:
#  sample of the dataset
n_samples = 4
sample_x, sample_y = x[:n_samples, :], y[:n_samples, :]

In [ ]:
#  activations & their derivatives

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def dsigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

def relu(x):
    return np.maximum(x, 0)

def drelu(x):
    x[x <= 0] = 0
    x[x > 0] = 1
    return x

In [ ]:
if activation == 'relu':
    act = relu
    dact = drelu
else:
    act = sigmoid
    dact = dsigmoid

def forward(x, w, b):
    #  input -> hidden
    z0 = x.dot(w[0]) + b[0]
    a0 = act(z0)

    #  hidden -> output
    pred = a0.dot(w[1]) + b[1]
    
    return z0, a0, pred

In [ ]:
z0, a0, pred = forward(sample_x, w, b)
pred

In [ ]:
def sq_error(pred, y):
    return 1/2 * np.power(pred - y, 2)

def abs_error(pred, y):
    return abs(pred - y)

def error(pred, y):
    return pred - y

In [ ]:
def backward(lr, reg, w, b, z0, a0, pred, y, x):
    derror = error(pred, y)
    
    dw1 = (a0.T).dot(derror)
    db1 = derror
    
    d1 = derror.dot(w[1].T) * dact(z0)
    dw0 = np.dot(x.T, d1)
    db0 = d1
    
    dw1 += reg * w[1]
    dw0 += reg * w[0]

    w[0] += -lr * dw0
    b[0] += -lr * np.sum(db0, 0, keepdims=True)
    w[1] += -lr * dw1
    b[1] += -lr * np.sum(db1, 0, keepdims=True)
    
    return w, b

In [ ]:
#  training
epochs = 10000
hist = []

for _ in range(epochs):
    z0, a0, pred = forward(x, w, b)
    hist.append(np.mean(abs_error(pred, y)))
    w, b = backward(lr, reg, w, b, z0, a0, pred, y, x)

In [ ]:
_, _, preds = forward(x, w, b)

In [ ]:
%matplotlib inline

plt.scatter(x, preds, label='pred', alpha=0.5)
plt.scatter(x, y, label='sampled y', alpha=0.1)
plt.plot(
    sorted(x),
    [y for _, y in sorted(zip(x, y_true))],
    label='true y', 
    color='black'
)
plt.legend()